In [0]:
# pyspark functions
from pyspark.sql.functions import *
# URL processing
import urllib

### Read AWS Credentials

In [0]:
# Define the path to the Delta table
delta_table_path = "dbfs:/user/hive/warehouse/authentication_credentials"

# Read the Delta table to a Spark DataFrame
aws_keys_df = spark.read.format("delta").load(delta_table_path)

In [0]:
# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

In [0]:
# AWS S3 bucket name
AWS_S3_BUCKET = "user-0a5040edb649-bucket"
# Mount name for the bucket
MOUNT_NAME = "/mnt/user-0a5040edb649-bucket"
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

## Read data from S3 bucket into dataframe

In [0]:
display(dbutils.fs.ls("mnt/user-0a5040edb649-bucket/topics"))

path,name,size,modificationTime
dbfs:/mnt/user-0a5040edb649-bucket/topics/0a5040edb649.geo/,0a5040edb649.geo/,0,1711491135720
dbfs:/mnt/user-0a5040edb649-bucket/topics/0a5040edb649.pin/,0a5040edb649.pin/,0,1711491135720
dbfs:/mnt/user-0a5040edb649-bucket/topics/0a5040edb649.user/,0a5040edb649.user/,0,1711491135720


In [0]:
%sql
-- Disable format checks during the reading of Delta tables 
SET spark.databricks.delta.formatCheck.enabled=false

key,value
spark.databricks.delta.formatCheck.enabled,false


In [0]:
topics = [".pin", ".geo", ".user"]

def read_topics_into_dataframe(topic):
    # File path for topic files
    file_path = f"/mnt/user-0a5040edb649-bucket/topics/0a5040edb649{topic}/partition=0/*.json"
    file_type = "json"
    # Ask Spark to infer the schema
    infer_schema = "true"
    # Read in JSONs from mounted S3 bucket to Spark dataframe
    df = spark.read.format(file_type) \
        .option("inferSchema", infer_schema) \
        .load(file_path)
    return df

for suffix in topics:
    make_df = f"df_{suffix[1:]} = read_topics_into_dataframe('{suffix}')"
    display_df = f"display(df_{suffix[1:]})"
    # execute statements
    exec(make_df)
    exec(display_df)


category,description,downloaded,follower_count,image_src,index,is_image_or_video,poster_name,save_location,tag_list,title,unique_id
christmas,"Christmas decorating ideas for porches. Beautiful holiday decor ideas for front porches both small and large. Outdoor decorations like sleds, lanterns, Christmas trees, wreaths,…",1,46k,https://i.pinimg.com/videos/thumbnails/originals/40/83/f5/4083f5b4971bf235f89a4784ab87271e.0000001.jpg,2482,video,"Life on Summerhill | Home, Holiday Decor & DIY Website",Local save in /data/christmas,"Diy Christmas Decorations For Home,Farmhouse Christmas Decor,Christmas Home,Christmas Holidays,Christmas Front Porches,How To Decorate For Christmas,Christmas Porch Ideas,Christmas Decorating Ideas,Large Outdoor Christmas Decorations",FORNT PORCH CHRISTMAS DECORATING IDEAS,08604f20-fa17-4b9a-9949-781717eca6cd
travel,"This Costa Rica itinerary is the ultimate guide to spending two weeks in Costa Rica. Find out about visiting La Fortuna, Arenal, Monteverde, Naranjo, Corcovado National Park, Or…",1,10k,https://i.pinimg.com/originals/30/93/cb/3093cb01d9de2d125fda8ba5e3e41946.jpg,10138,image,"Wanderlust Chloe ✈️ Travel guides, inspo and adventure travel ✈️",Local save in /data/travel,"Costa Rica Travel,Rio Celeste Costa Rica,Dream Vacations,Vacation Spots,Vacation Travel,Travel Pictures,Travel Photos,Fortuna Costa Rica,Costa Rica Pictures","14 Amazing Things To Do In Costa Rica | Volcanoes, Waterfalls, Wildlife And More",927c4658-cc3f-4b92-9b5c-70743d0c238d
travel,"This Costa Rica itinerary is the ultimate guide to spending two weeks in Costa Rica. Find out about visiting La Fortuna, Arenal, Monteverde, Naranjo, Corcovado National Park, Or…",1,10k,https://i.pinimg.com/originals/30/93/cb/3093cb01d9de2d125fda8ba5e3e41946.jpg,10138,image,"Wanderlust Chloe ✈️ Travel guides, inspo and adventure travel ✈️",Local save in /data/travel,"Costa Rica Travel,Rio Celeste Costa Rica,Dream Vacations,Vacation Spots,Vacation Travel,Travel Pictures,Travel Photos,Fortuna Costa Rica,Costa Rica Pictures","14 Amazing Things To Do In Costa Rica | Volcanoes, Waterfalls, Wildlife And More",927c4658-cc3f-4b92-9b5c-70743d0c238d
christmas,"My favorite 75+ Neutral Christmas Home Decor for decorating your house during the Holiday Season in earth tones and a farmhouse, rustic style all winter. I love this modern, sim…",1,31k,https://i.pinimg.com/originals/86/84/39/868439dd894969e3abd6a2a8a9fe1e9c.jpg,2604,image,Everyday Wholesome,Local save in /data/christmas,"Colorful Christmas Decorations,Colorful Christmas Tree,Christmas Centerpieces,Christmas Colors,Xmas Colors,Winter Decorations,Christmas Trends,Christmas Inspiration,Christmas Home",75+ Neutral Christmas Home Decor for the Holiday Season in Farmhouse Style using Earth Tones Modern,087b0fa9-f901-4262-aa0a-6caf234d1b35
diy-and-crafts,"This post may contain affiliate links, read our Disclosure Policy for more information. As an Amazon Associate I earn from qualifying purchases, thank you! Make some cute handpr…",1,892k,https://i.pinimg.com/originals/ff/fe/38/fffe384f3ec18a0d87cb2d80cc8c1499.jpg,3156,image,Michelle {CraftyMorning.com},Local save in /data/diy-and-crafts,"Christmas Gifts For Parents,Christmas Decorations For Kids,Christmas Crafts For Toddlers,Preschool Christmas,Christmas Crafts For Gifts,Christmas Activities,Toddler Crafts,Kids Christmas,Christmas Feeling",Handprint Reindeer Ornaments - Crafty Morning,fa6e31a4-18c2-4eca-a6d8-e903eee2c2a4
diy-and-crafts,"This post may contain affiliate links, read our Disclosure Policy for more information. As an Amazon Associate I earn from qualifying purchases, thank you! Make some cute handpr…",1,892k,https://i.pinimg.com/originals/ff/fe/38/fffe384f3ec18a0d87cb2d80cc8c1499.jpg,3156,image,Michelle {CraftyMorning.com},Local save in /data/diy-and-crafts,"Christmas Gifts For Parents,Christmas Decorations For Kids,Christmas Crafts For Toddlers,Preschool Christmas,Christmas Crafts For Gifts,Christmas Activities,Toddler Crafts,Kids Christmas,Chris

country,ind,latitude,longitude,timestamp
Antarctica (the territory South of 60 deg S),2418,-88.4642,-171.061,2022-05-27T11:30:59
Cocos (Keeling) Islands,10794,-89.5236,-154.567,2022-01-01T02:26:50
Central African Republic,2074,-52.3213,-50.11,2019-11-03T05:41:59
Antigua and Barbuda,7922,-88.0974,-172.052,2021-01-27T09:14:19
Dominican Republic,9979,14.9967,-120.682,2018-07-18T19:01:46
Cote d'Ivoire,2923,-84.6302,-164.507,2019-09-08T22:53:09
French Guiana,8304,-28.8852,-164.87,2019-09-13T04:50:29
Cote d'Ivoire,4315,-45.8508,66.1003,2019-12-15T03:51:28
Azerbaijan,5069,-63.0063,-157.474,2021-03-20T09:32:44
Mozambique,6145,-65.9079,-143.845,2019-12-05T02:09:44


age,date_joined,first_name,ind,last_name
21,2015-11-10T09:27:42,Andrea,8731,Alexander
24,2016-03-31T20:56:39,Austin,8887,Rodriguez
36,2015-12-20T16:38:13,Michelle,4315,Prince
32,2017-10-10T20:09:33,Christian,10625,Lang
22,2016-02-11T20:46:04,Jennifer,9672,Hudson
32,2016-04-02T03:51:23,Brittany,1313,Jones
34,2016-12-22T00:02:02,Thomas,10794,Turner
20,2016-01-07T19:49:22,David,2959,Griffith
26,2015-12-20T10:28:00,Brendan,9875,Joseph
21,2016-01-03T15:42:12,Annette,2074,Forbes
